In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) -->

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from google.cloud import storage
import pydicom
import itertools
import shutil

In [2]:
!pip install --upgrade scipy

  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/e0/9e/80e2205d138960a49caea391f3710600895dd8292b6868dc9aff7aa593f9/scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 37.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.11.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.11.0 which is incompatible.
libpysal 

In [3]:
!pip install git+https://github.com/wasserth/TotalSegmentator.git

  Cloning https://github.com/wasserth/TotalSegmentator.git to /tmp/pip-req-build-5zx7z4xf
  Running command git clone --filter=blob:none --quiet https://github.com/wasserth/TotalSegmentator.git /tmp/pip-req-build-5zx7z4xf
  Resolved https://github.com/wasserth/TotalSegmentator.git to commit 127f24323cf2378ccfa9c8c4da0682fd9417bd2d
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for dicom2nifti from https://files.pythonhosted.org/packages/81/af/32814e65c2d5726faaf3e90636e4f53bdf3253ead56ff05edfffd9b832da/dicom2nifti-2.4.9-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.8/151.8 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/6

In [4]:
from totalsegmentator.python_api import totalsegmentator


In [5]:
def get_blobs(bucket_name, prefix):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client(project = "CS-502")
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)
    return bucket, blobs

def get_bucket(bucket_name):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client(project = "CS-502")
    bucket = storage_client.get_bucket(bucket_name)
#     blobs = bucket.list_blobs(prefix=prefix)
    return bucket

In [6]:

def segment_imgs(blobs):
    """
    blobs: list of blobs for one subject and run
    
    """
    errors = [] 
    
    # temporary folder to save the files
    temp_source_folder = 'temp_source'
    if not os.path.exists(temp_source_folder):
        os.makedirs(temp_source_folder)

    temp_result_folder = 'temp_result'
    if not os.path.exists(temp_result_folder):
        os.makedirs(temp_result_folder)
    
    # download the files
#     filenames = []
    for blob in blobs:
        fn = '/'.join(blob.name.split('/')[1:-1])
        if not bucket.blob(os.path.join('train_segmented', fn, 'stomach.nii.gz')).exists():
            print(fn)
            if blob.name.endswith('.dcm'):
#                 filenames.append(fn)
                blob.download_to_filename(os.path.join(temp_source_folder, blob.name.split('/')[-1]))
    
    try:
        totalsegmentator(temp_source_folder, temp_result_folder, roi_subset = ["spleen", "kidney_left", "kidney_right", "liver", "esophagus", "colon", "duodenum", "small_bowel", "stomach"])
    except:
        errors.append(fn)
    
    # upload the files
    for i, filename in enumerate(os.listdir(temp_result_folder)):
        output_path = os.path.join('train_segmented', fn, filename)
        bucket.blob(output_path).upload_from_filename(os.path.join(temp_result_folder, filename))
    
    # delete temp files
    shutil.rmtree(temp_source_folder)
    shutil.rmtree(temp_result_folder)
    
    return errors
    

In [8]:
# shutil.rmtree('/kaggle/working/temp_source')
# shutil.rmtree('/kaggle/working/temp_result')
    

In [9]:
bucket, all_blobs = get_blobs('rsna-competition-2023', 'train_images_fixed/train_images')
blobs_sep = [list(blobs) for _, blobs in itertools.groupby(all_blobs, lambda blob: blob.name.split('/')[2:4])]


In [ ]:
all_errors = []
for run in blobs_sep[1:]:
    print(len(run))
    err = segment_imgs(run)
    all_errors.append(err)
    print(' ################################# Run done: ', run[0].name.split('/')[2:4], '#################################')

105

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

TotalSegmentator sends anonymous usage statistics. If you want to disable it check the documentation.


<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Downloading: 100%|██████████| 234M/234M [00:00<00:00, 377MB/s] 


Download finished. Extracting...


Downloading: 100%|██████████| 234M/234M [00:00<00:00, 486MB/s] 


...

  Predicted in 7.30s
Resampling...
Converting dicom to nifti...
  found image with shape (512, 512, 24)
  cropping from (512, 512, 24) to (380, 287, 21)
Resampling...
  Resampled in 3.75s
Predicting part 1 of 1 ...


<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


  Predicted in 15.93s
Resampling...
Saving segmentations...


  0%|          | 0/9 [00:00<?, ?it/s]

  Saved in 1.26s
 ################################# Run done:  ['24499', '21914'] #################################
9
train_images/24499/53946
train_images/24499/53946
train_images/24499/53946
train_images/24499/53946
train_images/24499/53946
train_images/24499/53946
train_images/24499/53946
train_images/24499/53946
train_images/24499/53946

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough body segmentation...
Converting dicom to nifti...
  found image with shape (512, 512, 9)
Resampling...
  Resampled in 0.16s
Predicting...
